In [13]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from pathlib import Path
import shutil

In [14]:
def removeIdentifierFile(path):
    for file in os.listdir(path):
        if file.find(".Identifier") != -1:
            os.remove(os.path.join(path, file))

In [15]:
import os, random

raw_imposters_folder = "new_imposter"
raw_valid_user_folder = "valid_user"

removeIdentifierFile(raw_imposters_folder)
removeIdentifierFile(raw_valid_user_folder)

img_count = 500 # img taken from each class

imposter_filenames = random.sample(os.listdir(raw_imposters_folder), img_count)
valid_user_filenames = random.sample(os.listdir(raw_valid_user_folder), img_count)

train_valid_user_folder = "train/valid_user"
train_imposters_folder = "train/imposters"

Path(train_valid_user_folder).mkdir(parents=True, exist_ok=True)
Path(train_imposters_folder).mkdir(parents=True, exist_ok=True)

test_valid_user_folder = "test/valid_user"
test_imposters_folder = "test/imposters"

Path(test_valid_user_folder).mkdir(parents=True, exist_ok=True)
Path(test_imposters_folder).mkdir(parents=True, exist_ok=True)


In [16]:
class_training_size = int(img_count * 0.9)

count = 0
for imposter_filename in imposter_filenames[:class_training_size]:
    shutil.copy(os.path.join(raw_imposters_folder, imposter_filename), os.path.join(train_imposters_folder, imposter_filename))
    count += 1

count = 0
for imposter_filename in imposter_filenames[class_training_size:]:
    shutil.copy(os.path.join(raw_imposters_folder, imposter_filename), os.path.join(test_imposters_folder, imposter_filename))


for valid_user_filename in valid_user_filenames[:class_training_size]:
    shutil.copy(os.path.join(raw_valid_user_folder, valid_user_filename), os.path.join(train_valid_user_folder, valid_user_filename))

for valid_user_filename in valid_user_filenames[class_training_size:]:
    shutil.copy(os.path.join(raw_valid_user_folder, valid_user_filename), os.path.join(test_valid_user_folder, valid_user_filename))

removeIdentifierFile(train_imposters_folder)
removeIdentifierFile(test_imposters_folder)
removeIdentifierFile(train_valid_user_folder)
removeIdentifierFile(test_valid_user_folder)


In [17]:
train = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)

train_dataset = train.flow_from_directory("train",
    target_size=(150,150),
    batch_size = 32,
    class_mode = 'binary')
                                         
test_dataset = test.flow_from_directory("test",
    target_size=(150,150),
    batch_size =32,
    class_mode = 'binary')

Found 900 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [18]:
test_dataset.class_indices


{'imposters': 0, 'valid_user': 1}

In [19]:
model = keras.Sequential()

model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(512,activation='relu'))

model.add(keras.layers.Dense(1,activation='sigmoid'))

In [20]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [21]:
model.fit_generator(train_dataset,
    steps_per_epoch = 250,
    epochs = 10,
    validation_data = test_dataset
)

model.save('classifier')

Epoch 1/10


/tmp/ipykernel_16433/1593974248.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_dataset,


 29/250 [==>...........................] - ETA: 1:16 - loss: 0.1711 - accuracy: 0.9389WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 2500 batches). You may need to use the repeat() function when building your dataset.


250/250 [==============================] - 13s 41ms/step - loss: 0.1711 - accuracy: 0.9389 - val_loss: 3.6751e-05 - val_accuracy: 1.0000


INFO:tensorflow:Assets written to: classifier/assets


INFO:tensorflow:Assets written to: classifier/assets


In [22]:
def predictImage(filename):
    img1 = image.load_img(filename,target_size=(150,150))
    model = tf.keras.models.load_model('classifier')
 
    Y = image.img_to_array(img1)
    
    X = np.expand_dims(Y,axis=0)
    val = model.predict(X, verbose=0)
    return val[0][0]




In [23]:
test_imposter = "test/imposters"
test_valid_user = "test/valid_user"
correct_count = 0
wrong_count = 0
count = 0

for file in os.listdir(test_imposter):
    id = predictImage(os.path.join(test_imposter, file))
    count += 1
    if id == 0:
        correct_count += 1
        print(f"{file} correct {id}")
    else:
        wrong_count += 1
        print(f"{file} wrong {id}")


for file in os.listdir(test_valid_user):
    id = predictImage(os.path.join(test_valid_user, file))
    count += 1
    if id == 1:
        correct_count += 1
        print(f"{file} correct {id}")
    else:
        wrong_count += 1
        print(f"{file} wrong {id}")

60.png correct 0.0
84.png correct 0.0
209.png correct 0.0
366.png correct 0.0
524.png correct 0.0
490.png correct 0.0
302.png correct 0.0
129.png correct 0.0
575.png correct 0.0
350.png correct 0.0
76.png correct 0.0
30.png correct 0.0
298.png correct 0.0
281.png correct 0.0
597.png correct 0.0
372.png correct 0.0
109.png correct 0.0
125.png correct 0.0
244.png correct 0.0
478.png correct 0.0
530.png correct 0.0
483.png correct 0.0
54.png correct 0.0
14.png correct 0.0
67.png correct 0.0
568.png correct 0.0
441.png correct 0.0
592.png correct 0.0
294.png correct 0.0
376.png correct 0.0
361.png correct 0.0
454.png correct 0.0
534.png correct 0.0
37.png correct 0.0
521.png correct 0.0
26.png correct 0.0
64.png correct 0.0
117.png correct 0.0
507.png correct 0.0
137.png correct 0.0
90.png correct 0.0
313.png correct 0.0
532.png correct 0.0
23.png correct 0.0
49.png correct 0.0
408.png correct 0.0
257.png correct 0.0
216.png correct 0.0
519.png correct 0.0
509.png correct 0.0
580.png corre

In [24]:
print(correct_count / count)


1.0
